# INIT

## Librerias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importacion datos

In [4]:
# Diccionario colecciones
db_collections = [
    'calendar',
    'items',
    'prices',
    'tiendas',
    'departments',
    'sales_main',
    'weekly_sales',
    'year_sales',
    'day_sales'
    ]

### Mongo DB

In [1]:
'''
from modules.mongo_local_module import connect_to_db, get_collection_as_dataframe

db_name = 'ds_market'         # Nombre de la base de datos

# Conectar a la base de datos
db = connect_to_db(db_name, local=True)

# Obtener colecciones
df_cal = get_collection_as_dataframe(db, 'calendar').drop(columns='_id')
# Se añadieron columnas week y month a calendar para hacer cruces de datos mas faciles

df_cal = df_cal.fillna('None') # event tiene NaN



df_shops = get_collection_as_dataframe(db, 'tiendas').drop(columns='_id')
df_items = get_collection_as_dataframe(db, 'items').drop(columns='_id')

df_ws = get_collection_as_dataframe(db, 'weekly_sales').drop(columns='_id')

# Añadimos eventos para no perder esa información
# Crear un diccionario desde df_cal
event_dict = df_cal.set_index('week')['event'].to_dict()

# Añadir la columna 'event' a df_ws usando el método 'map'
df_ws['event'] = df_ws['week'].map(event_dict)
'''

"\nfrom modules.mongo_local_module import connect_to_db, get_collection_as_dataframe\n\ndb_name = 'ds_market'         # Nombre de la base de datos\n\n# Conectar a la base de datos\ndb = connect_to_db(db_name, local=True)\n\n# Obtener colecciones\ndf_cal = get_collection_as_dataframe(db, 'calendar').drop(columns='_id')\n# Se añadieron columnas week y month a calendar para hacer cruces de datos mas faciles\n\ndf_cal = df_cal.fillna('None') # event tiene NaN\n\n\n\ndf_shops = get_collection_as_dataframe(db, 'tiendas').drop(columns='_id')\ndf_items = get_collection_as_dataframe(db, 'items').drop(columns='_id')\n\ndf_ws = get_collection_as_dataframe(db, 'weekly_sales').drop(columns='_id')\n\n# Añadimos eventos para no perder esa información\n# Crear un diccionario desde df_cal\nevent_dict = df_cal.set_index('week')['event'].to_dict()\n\n# Añadir la columna 'event' a df_ws usando el método 'map'\ndf_ws['event'] = df_ws['week'].map(event_dict)\n"

### CSVs

In [1]:
from modules.get_data import get_data

In [2]:
df_cal = get_data('exp_calendar.csv')
df_items = get_data('exp_items.csv')
df_shops = get_data('exp_shops.csv')
df_ws = get_data('exp_weekly_sales.csv')

In [3]:
display(df_cal.head())
display(df_items.head())
display(df_shops.head())
display(df_ws.head())

,fecha,event
0,2011-02-06,SuperBowl
1,2011-08-01,Ramadan starts
2,2011-11-24,Thanksgiving
3,2012-01-01,NewYear
4,2012-02-05,SuperBowl


,item,category,department
0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1
1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1
2,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1
3,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1
4,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1


,store_code,region,store
0,NYC_1,New York,Greenwich_Village
1,NYC_2,New York,Harlem
2,NYC_3,New York,Tribeca
3,NYC_4,New York,Brooklyn
4,BOS_1,Boston,South_End


,item,store_code,ejercicio,semana,unidades,sell_price,cifra
0,ACCESORIES_1_004,BOS_1,2011,4,3.0,5.77,17.32
1,ACCESORIES_1_004,BOS_2,2011,4,8.0,5.77,46.18
2,ACCESORIES_1_004,NYC_2,2011,4,9.0,5.77,51.95
3,ACCESORIES_1_004,NYC_4,2011,4,2.0,5.77,11.54
4,ACCESORIES_1_004,PHI_1,2011,4,3.0,5.77,17.32


# **Creacion caracteristicas**
___
___  
  
Creamos las caracteristicas generales y luego filtraremos para cada supuesto.

In [ ]:
from modules import feature_creation_module as fcm

Resumen de las **Primitivas de Agregación** y **Transformación**:

**Primitivas de Agregación:**

1\. **`sum`**: 

   - **Por qué la usamos**: Nos permite calcular el total de unidades vendidas y los ingresos (`raw_earn`) generados por cada producto.

   - **Qué esperamos obtener**: Queremos conocer cuáles productos generan mayores ingresos o tienen mayores volúmenes de venta en total, lo cual es un indicador de rendimiento absoluto.

2\. **`mean`**:

   - **Por qué la usamos**: El promedio de ventas (`units`) y el precio de venta (`sell_price`) por semana nos ayuda a ver qué tan consistente es el rendimiento de un producto a lo largo del tiempo.

   - **Qué esperamos obtener**: Identificar productos con un rendimiento constante o con precios estables, lo cual puede ser útil para determinar qué productos son menos volátiles en términos de ventas o precio.

3\. **`std`**:

   - **Por qué la usamos**: La desviación estándar mide la variabilidad en ventas y en ingresos. Es útil para entender la estabilidad o fluctuación de los productos.

   - **Qué esperamos obtener**: Productos con alta desviación estándar tienen ventas más impredecibles, mientras que aquellos con baja desviación estándar son más consistentes. Esto ayuda a identificar productos estables frente a aquellos que experimentan picos o caídas drásticas.

4\. **`count`**:

   - **Por qué la usamos**: Nos ayuda a saber en cuántas tiendas se vendió cada producto.

   - **Qué esperamos obtener**: Determina la penetración del producto en el mercado. Productos que se venden en más tiendas podrían ser más populares o estar más ampliamente disponibles.

5\. **`max`**:

   - **Por qué la usamos**: Queremos conocer el valor máximo de ventas (`units`) o ingresos (`raw_earn`) de un producto en una semana.

   - **Qué esperamos obtener**: Identificar productos que hayan tenido semanas excepcionales de ventas o ingresos, lo que puede indicar eventos o promociones que impulsaron el rendimiento.

---

**Primitivas de Transformación:**

1\. **`divide_numeric`**:

   - **Por qué la usamos**: Calcula relaciones como `units/sell_price`, lo que nos permite obtener el ratio de unidades vendidas por cada unidad de precio.

   - **Qué esperamos obtener**: Nos da una idea de cómo varía la relación entre el precio y las ventas. Por ejemplo, productos que venden muchas unidades a precios más bajos podrían comportarse de manera diferente a aquellos que venden menos unidades a precios altos.

2\. **`percentile`**:

   - **Por qué la usamos**: Nos permite calcular el percentil de un valor en una distribución (por ejemplo, cómo se comparan las ventas de un producto con el resto).

   - **Qué esperamos obtener**: Clasificar productos según su rendimiento en comparación con los demás. Esto nos ayuda a identificar cuáles están en los niveles más altos o bajos de rendimiento relativo.

---

¿Qué esperamos obtener con estas características?

1\. **Identificación de patrones de ventas**: 

   - Queremos descubrir qué productos se venden de forma constante (baja variabilidad) frente a aquellos que son más volátiles (alta desviación estándar en ventas o ingresos).

2\. **Estudio del precio frente a las ventas**:

   - Al calcular ratios como `units/sell_price`, buscamos entender si el precio influye en el volumen de ventas. Productos con un bajo precio podrían compensar con altos volúmenes de venta y viceversa.

En resumen, estas características nos ayudarán a realizar un **clustering más efectivo**, diferenciando productos con base en su rendimiento en ventas, su consistencia, su presencia en distintas tiendas y cómo reaccionan a factores como el precio o el tiempo.

In [ ]:
# # Creacion caracteristicas para productos basadas en ventas/performance
# fc_config = {
#     'target_df': 'items',  # Objetivo: productos
#     'agg_primitives': [
#         'sum',      # Total de ventas, ingresos
#         'mean',     # Promedio de ventas, precio, ingresos
#         'std',      # Variabilidad en ventas e ingresos
#         'count'     # Total de tiendas donde se vendió
#     ],  
#     'trans_primitives': [],
#     'max_depth': 2
# }

# Creamos caracteristicas a lo bruto para filtrar más tarde
# fc_config = {
#     'target_df': 'items',  # Queremos generar características sobre los productos (items)
#     'agg_primitives': [
#         'sum',       # Total de ventas, ingresos
#         'mean',      # Promedio de ventas, precio, ingresos
#         'std',       # Variabilidad en ventas e ingresos
#         'count',     # Total de tiendas donde se vendió
#         'max',       # Máximo para calcular ventas máximas en una tienda
#     ],
#     'trans_primitives': [
#         # 'divide_numeric',  # Ratio de ventas por ingreso, proporción de disponibilidad
#         'percentile'       # Porcentaje de ventas respecto a la tienda de mayor rendimiento
#     ],
#     'max_depth': 2
# }
fc_config = {
    'target_df': 'sales',  # Nombre del DataFrame objetivo
    'agg_primitives': [
        'min',               # Mínimo
        'max',               # Máximo
        'mean',              # Promedio
        'std',               # Desviación estándar
        'sum',               # Suma
        'count'              # Totales
    ],
    'trans_primitives': [
        # 'divide_numeric',    # División entre características numéricas -> GENERA MUCHAS!!!
        'percentile'       # Porcentaje de ventas respecto a la tienda de mayor rendimiento
    ],
    'max_depth': 2         # Profundidad máxima de la generación de características
}


feature_matrix, feature_defs = fcm.create_features(df_items, df_shops, df_ws, fc_config)

In [ ]:
len(feature_defs)

In [ ]:
# Revision definiciones caracteristicas creadas
fcm.get_features_descriptions(feature_defs)

# **PRODUCTOS: PERFORMANCE**
___
___

__Filtrado caracteristicas usando liberira FeatureTools__
**Feature encode** (codifica categorical)

> https://featuretools.alteryx.com/en/stable/api_reference.html#feature-encoding

**Feature selection**

> https://featuretools.alteryx.com/en/stable/api_reference.html#feature-selection  
  
- *remove_low_information_features(feature_matrix)*  
Select features that have at least 2 unique values and that are not all null

- *remove_highly_correlated_features(feature_matrix)*  
Removes columns in feature matrix that are highly correlated with another column.

- *remove_highly_null_features(feature_matrix)*  
Removes columns from a feature matrix that have higher than a set threshold of null values.

- *remove_single_value_features(feature_matrix)*  
Removes columns in feature matrix where all the values are the same.

In [ ]:
# Aplicar filtros a las caracteristicas creadas
# Remove low information, highly correlated, highly null and single value features
filtered_feature_matrix = fcm.filter_feature_matrix(feature_matrix)

display(filtered_feature_matrix.head(5))

In [51]:
# Seleccionar y renombrar características relevantes para el clustering de productos ventas/performance
# selected_features = {
#     'COUNT(sales)': 'num_sales',                   
#     'SUM(sales.units)': 'total_units_sold',        
#     'SUM(sales.raw_earn)': 'total_revenue',        
#     'MEAN(sales.sell_price)': 'avg_sell_price',    
#     'STD(sales.sell_price)': 'std_sell_price',     
#     'STD(sales.units)': 'std_units_sold',          
#     'MEAN(sales.units)': 'avg_units_per_week',     
#     'MEAN(sales.week)': 'avg_week',                
#     'STD(sales.week)': 'std_week',                 
#     'MEAN(sales.year)': 'avg_year'                 
# }

selected_features = {
    'COUNT(sales)': 'num_sales',                      # Número total de ventas
    'SUM(sales.raw_earn)': 'total_revenue',           # Suma de las ganancias brutas
    'MEAN(sales.sell_price)': 'avg_sell_price',       # Precio de venta promedio
    'MAX(sales.units)': 'max_units_sold',             # Máximo número de unidades vendidas
    'STD(sales.raw_earn)': 'std_revenue',             # Desviación estándar de las ganancias brutas
    'STD(sales.sell_price)': 'std_sell_price',        # Desviación estándar del precio de venta
    'PERCENTILE(MAX(sales.raw_earn))': 'percentile_max_revenue', # Percentil del máximo de ganancias brutas
    'PERCENTILE(MEAN(sales.sell_price))': 'percentile_avg_sell_price' # Percentil del promedio del precio de venta
}

feature_matrix_selected = fcm.select_features(feature_matrix, selected_features)

In [52]:
# Renombrar características si es necesario
# feature_matrix_renamed = fcm.rename_features(filtered_feature_matrix, selected_features) # Hemos filtrado antes de renombrar
feature_matrix_renamed = fcm.rename_features(feature_matrix_selected, selected_features)

In [ ]:
feature_matrix_renamed.head(5)

## Preprocesamiento de los datos

In [54]:
from modules import preprocessing_module as pm

In [ ]:
scaled_df = pm.preprocess_features(feature_matrix_renamed)

In [ ]:
scaled_df.head()

## Kmeans

In [57]:
from modules import kmeans_module as km_m

### Elbow & Silouette

In [ ]:
km_m.plot_elbow_silhouette(scaled_df, 30)

In [ ]:
# Apreciamos varias opciones 5 y 10
optimal_k = 5 

kmeans_model, feature_matrix_with_clusters = km_m.apply_kmeans_and_plot(optimal_k, scaled_df, feature_matrix_renamed)

feature_matrix_with_clusters.head(5)

In [ ]:
feature_matrix_with_clusters.shape

### Analisis

#### PCAs

In [ ]:
pca_df = km_m.pca_visualization_2d(feature_matrix_with_clusters)

display(pca_df.head())

In [ ]:
explained_variance = km_m.pca_variance_plot(feature_matrix_with_clusters, 1)

display(explained_variance)

#### Importancia caracteristicas

In [ ]:
# Preprocesamos datos
cluster_col='Cluster'
cols_to_scale = feature_matrix_with_clusters.columns.tolist()
df_clusters = feature_matrix_with_clusters[[cluster_col]].copy().reset_index(drop=True)
cols_to_scale.remove(cluster_col) # Columna Objetivo
preprocessed_df = pm.preprocess_features(feature_matrix_with_clusters[cols_to_scale])

# Combinar los DataFrames basándose en la columna 'Index'
preprocessed_df = pd.concat([preprocessed_df, df_clusters], axis=1)

clusters = kmeans_model.labels_

# sales_importance_df = km_m.get_feature_importances(feature_matrix_with_clusters, clusters, imp_threshold=0.05)
sales_importance_df = km_m.get_feature_importances(preprocessed_df, clusters, imp_threshold=0.05)

display(sales_importance_df.head(10))

### Dimension Redux

In [ ]:
pca_df, importance_df = km_m.redux_dimensions_pca_and_cluster(preprocessed_df, n_clusters=optimal_k, n_components=1)

In [ ]:
# El segundo componente es residual
pca_df, importance_df = km_m.redux_dimensions_pca_and_cluster(preprocessed_df, n_clusters=optimal_k, n_components=2)

## Descripcion de los clusters

In [ ]:
# Ejecutar PCA y clustering
# optimal_k = 10
n_components = 1
df_with_pca, importance_df = km_m.pca_and_cluster(scaled_df, n_clusters=optimal_k, n_components=n_components)

# Mostrar el DataFrame con PCA y clustering
display(df_with_pca.head(5))

# Graficar la cantidad de items por cluster
# km_m.plot_items_per_cluster(df_with_pca)
show_clusters = km_m.plot_cluster_sizes(df_with_pca)

### Mostrar descripciones

In [ ]:
# Crear y mostrar descripciones de los clusters
description_df = km_m.create_cluster_descriptions(df_with_pca, importance_df)
print('###########################################################')
print('Descripcion de los clusters')
print('###########################################################')
display(description_df)

Para nombrar y describir los clusters en función de las estadísticas proporcionadas para cada característica, considera lo siguiente:

1. **Interpretación de Estadísticas**

Aquí está una interpretación básica de cada característica basada en los valores proporcionados para cada cluster:

- **Max Units Sold**: La cantidad máxima de unidades vendidas. Los valores varían desde negativos (menos unidades) hasta positivos (más unidades).

- **Std Revenue**: La desviación estándar de los ingresos totales. Indica la variabilidad en los ingresos.

- **Total Revenue**: Los ingresos totales generados por los productos. Valores positivos indican mayores ingresos.

- **Percentile Max Revenue**: Percentil del máximo ingreso. Mide cómo se compara el máximo ingreso de cada producto con otros productos.

- **Num Sales**: El número total de ventas realizadas. Los valores varían desde negativos (menos ventas) hasta positivos (más ventas).

2. **Asignación de Nombres y Descripción a Clusters**

Voy a usar los valores de cada característica para describir y nombrar cada cluster. Los valores negativos y positivos sugieren que los clusters pueden tener diferentes perfiles basados en estas características.

**Cluster 0**

- **Nombre**: **Bajo Rendimiento General**

- **Descripción**: Este cluster se caracteriza por una baja cantidad máxima de unidades vendidas y bajos ingresos totales y estándar. Las ventas y los ingresos se encuentran en los percentiles más bajos. Es el cluster con menor actividad general y menor variabilidad en los ingresos.

**Cluster 1**

- **Nombre**: **Ventas Moderadas con Ingresos Altos**

- **Descripción**: Este cluster muestra un número moderado de ventas y un ingreso total bajo en comparación con otros clusters. Sin embargo, la desviación estándar en los ingresos es alta, lo que sugiere variabilidad. Los ingresos máximos están en un percentil bajo, indicando que los productos no están generando ingresos máximos comparables a otros clusters.

**Cluster 2**

- **Nombre**: **Alto Ingreso y Ventas Consistentes**

- **Descripción**: Aquí encontramos un alto nivel en la cantidad máxima de unidades vendidas y en los ingresos totales. La desviación estándar en los ingresos es también alta, lo que indica una considerable variabilidad en los ingresos. Los productos en este cluster generan ingresos totales consistentes con percentiles altos.

**Cluster 3**

- **Nombre**: **Ventas Altas y Alta Variabilidad en Ingresos**

- **Descripción**: Este cluster se distingue por una gran cantidad máxima de unidades vendidas y una alta desviación estándar en los ingresos. Los ingresos totales son elevados, con percentiles máximos también altos. La alta variabilidad sugiere que los productos tienen un alto rendimiento con grandes fluctuaciones en los ingresos.

**Cluster 4**

- **Nombre**: **Ventas Moderadas y Alta Rentabilidad**

- **Descripción**: Este cluster tiene un número moderado de ventas y un alto ingreso total, con baja variabilidad en los ingresos. El percentil de ingresos máximos es alto, indicando que los productos en este cluster tienen un buen rendimiento en términos de ingresos, pero con menos variabilidad comparado con el cluster 3.

### DF Descripciones

In [ ]:
# Crear el DataFrame
data = {
    'Cluster': [0, 1, 2, 3, 4],
    'Nombre': [
        'Bajo Rendimiento General',
        'Ventas Moderadas con Ingresos Altos',
        'Alto Ingreso y Ventas Consistentes',
        'Ventas Altas y Alta Variabilidad en Ingresos',
        'Ventas Moderadas y Alta Rentabilidad'
    ],
    'Descripción': [
        'Bajo número de ventas, bajos ingresos y baja variabilidad.',
        'Ventas moderadas, ingresos bajos pero alta variabilidad.',
        'Alta cantidad de ventas, altos ingresos y alta variabilidad.',
        'Alta cantidad de ventas e ingresos con gran variabilidad.',
        'Ventas moderadas, altos ingresos con baja variabilidad.'
    ]
}

df_clusters = pd.DataFrame(data)

# Mostrar el DataFrame
display(df_clusters)


## Asignacion clusters a items

# **PRODUCTOS: PRECIOS**
___
___

**Rangos de precios de productos**

In [69]:
# # Crear la feature matrix para los productos (items) enfocada en los precios
# feature_matrix_price, feature_defs_price = ft.dfs(
#     entityset=es,
#     target_dataframe_name="items",
#     agg_primitives=["mean", "std", "min", "max"],
#     trans_primitives=[],  # No necesitamos transformaciones adicionales aquí
#     max_depth=2
# )

# # Seleccionar y renombrar características relevantes para el análisis de precios
# selected_features_price = {
#     'MEAN(sales.sell_price)': 'avg_sell_price',  # Precio promedio de venta
#     'STD(sales.sell_price)': 'std_sell_price',   # Desviación estándar del precio de venta
#     'MIN(sales.sell_price)': 'min_sell_price',   # Precio mínimo de venta
#     'MAX(sales.sell_price)': 'max_sell_price',   # Precio máximo de venta
# }

## Creacion caracteristicas

In [ ]:
# display(feature_matrix.shape)

feature_matrix.columns.tolist()

**Explicación**

1.  **`MEAN(sales.sell_price)`**: Mide el precio medio de los productos. Es crucial para determinar el rango promedio de precios.

2.  **`MAX(sales.sell_price)`**: Identifica el precio máximo, lo que ayuda a entender el rango superior de precios en el dataset.

3.  **`STD(sales.sell_price)`**: Mide la variabilidad en los precios. Una alta desviación estándar indica una amplia gama de precios, lo cual es relevante para el clustering.

4.  **`PERCENTILE(MAX(sales.sell_price))`**: Mide la posición relativa del máximo precio en comparación con otros productos. Ayuda a comprender la distribución extrema de los precios.

5.  **`PERCENTILE(MEAN(sales.sell_price))`**: Mide la posición relativa del promedio de precios. Ofrece perspectiva sobre cómo se compara el promedio de precios con otros productos.

6.  **`PERCENTILE(STD(sales.sell_price))`**: Mide la posición relativa de la desviación estándar en comparación con otros productos. Proporciona información sobre la variabilidad relativa de los precios.

**¿Por Qué Seleccionar Estas Características?**

Estas características permiten una comprensión integral de los precios en tus datos:

-   **Promedio y Máximo**: Para entender los rangos generales y extremos de precios.
-   **Desviación Estándar**: Para conocer la variabilidad de precios.
-   **Percentiles**: Para comparar cómo se sitúan los precios en relación con el resto de los datos.

Seleccionar estas características permitirá una segmentación efectiva en clusters basados en rangos de precios, capturando tanto la variabilidad como la posición relativa de los precios.

In [ ]:
price__features = {
    'MEAN(sales.sell_price)': 'avg_sell_price',   # Promedio del precio de venta, útil para conocer el rango medio de precios.
    'MAX(sales.sell_price)': 'max_sell_price',     # Máximo precio de venta, ayuda a identificar los valores más altos en el rango de precios.
    'STD(sales.sell_price)': 'std_sell_price',     # Desviación estándar del precio de venta, muestra la variabilidad en los precios.
    'PERCENTILE(MAX(sales.sell_price))': 'percentile_max_sell_price',  # Percentil del máximo precio de venta, muestra la posición relativa del precio máximo.
    'PERCENTILE(MEAN(sales.sell_price))': 'percentile_avg_sell_price',  # Percentil del promedio del precio de venta, muestra la posición relativa del promedio de precios.
    'PERCENTILE(STD(sales.sell_price))': 'percentile_std_sell_price'    # Percentil de la desviación estándar del precio de venta, muestra la variabilidad en comparación con otros precios.
}

feature_matrix_selected = fcm.select_features(feature_matrix, price_features)

# Renombrar características si es necesario
# feature_matrix_renamed = fcm.rename_features(filtered_feature_matrix, selected_features) # Hemos filtrado antes de renombrar
# feature_matrix_renamed = fcm.rename_features(feature_matrix_selected, price_features)

# display(feature_matrix_renamed.head(5))

## Preprocesamiento

In [ ]:
scaled_df = pm.preprocess_features(feature_matrix_renamed)

In [ ]:
scaled_df.head(5)

## Kmeans

### Elbow & Silhoutte

In [ ]:
km_m.plot_elbow_silhouette(scaled_df, 30)

### Analisis

#### PCAs

In [ ]:
pca_df = km_m.pca_visualization_2d(feature_matrix_with_clusters)

display(pca_df.head())

In [ ]:
explained_variance = km_m.pca_variance_plot(feature_matrix_with_clusters, 1)

display(explained_variance)

#### Importancia caracteristicas

In [ ]:
# Preprocesamos datos
cluster_col='Cluster'
cols_to_scale = feature_matrix_with_clusters.columns.tolist()
df_clusters = feature_matrix_with_clusters[[cluster_col]].copy().reset_index(drop=True)
cols_to_scale.remove(cluster_col) # Columna Objetivo
preprocessed_df = pm.preprocess_features(feature_matrix_with_clusters[cols_to_scale])

# Combinar los DataFrames basándose en la columna 'Index'
preprocessed_df = pd.concat([preprocessed_df, df_clusters], axis=1)

clusters = kmeans_model.labels_

# sales_importance_df = km_m.get_feature_importances(feature_matrix_with_clusters, clusters, imp_threshold=0.05)
sales_importance_df = km_m.get_feature_importances(preprocessed_df, clusters, imp_threshold=0.05)

display(sales_importance_df.head(10))

### Dimension Redux

In [ ]:
pca_df, importance_df = km_m.redux_dimensions_pca_and_cluster(preprocessed_df, n_clusters=optimal_k, n_components=1)

In [ ]:
# El segundo componente es residual
pca_df, importance_df = km_m.redux_dimensions_pca_and_cluster(preprocessed_df, n_clusters=optimal_k, n_components=2)

## Descripcion de los clusters

### Mostrar descripciones

In [ ]:
# Crear y mostrar descripciones de los clusters
description_df = km_m.create_cluster_descriptions(df_with_pca, importance_df)
print('###########################################################')
print('Descripcion de los clusters')
print('###########################################################')
display(description_df)

Lipsum dolor sit amet

### DF Descripciones

In [ ]:
# Crear el DataFrame
data = {
    'Cluster': [0, 1, 2, 3, 4],
    'Nombre': [
        'Bajo Rendimiento General',
        'Ventas Moderadas con Ingresos Altos',
        'Alto Ingreso y Ventas Consistentes',
        'Ventas Altas y Alta Variabilidad en Ingresos',
        'Ventas Moderadas y Alta Rentabilidad'
    ],
    'Descripción': [
        'Bajo número de ventas, bajos ingresos y baja variabilidad.',
        'Ventas moderadas, ingresos bajos pero alta variabilidad.',
        'Alta cantidad de ventas, altos ingresos y alta variabilidad.',
        'Alta cantidad de ventas e ingresos con gran variabilidad.',
        'Ventas moderadas, altos ingresos con baja variabilidad.'
    ]
}

df_clusters = pd.DataFrame(data)

# Mostrar el DataFrame
display(df_clusters)


## Asignacion clusters a items

# **PRODUCTOS: EVENTOS**
___
___

**Efecto de eventos en ventas de productos**

In [ ]:
# # Crear la feature matrix para los productos (items) enfocada en el efecto de los eventos
# feature_matrix_events, feature_defs_events = ft.dfs(
#     entityset=es,
#     target_dataframe_name="items",
#     agg_primitives=["mean", "count", "sum"],
#     trans_primitives=[],  # No necesitamos transformaciones adicionales aquí
#     max_depth=2
# )

# # Seleccionar y renombrar características relevantes para el análisis del efecto de eventos
# selected_features_events = {
#     'MEAN(sales.event)': 'avg_event_sales_ratio',  # Promedio de ventas durante eventos
#     'COUNT(sales.event)': 'num_events',            # Número de eventos en los que el producto fue vendido
#     'SUM(sales.units)': 'total_units_sold',        # Total de unidades vendidas
#     'SUM(sales.raw_earn)': 'total_revenue',        # Ingresos totales generados
# }

## Creacion caracteristicas

## Preprocesamiento

## Kmeans

## Descripcion de los clusters

## Asignacion clusters a items

___
___

# SECCION - master

**Descripcion** Lipsum dolor sit amet

## Creacion caracteristicas

## Preprocesamiento

## Kmeans

## Descripcion de los clusters

___
___

# CLUSTERING MODULE

In [25]:
from modules import clustering_module as cm

In [ ]:
# K-means
rf_model_kmeans, preprocessor_kmeans, pca_loadings_kmeans, pca_contributions_kmeans = cm.main(
    feature_matrix_renamed, method='kmeans', n_clusters=4
)

In [ ]:
# K-means
rf_model_kmeans, preprocessor_kmeans, pca_loadings_kmeans, pca_contributions_kmeans = cm.main(
    preprocessed_df, method='kmeans', n_clusters=optimal_k
)

In [ ]:
kmeans_cluster_labels, kmeans_cluster_summary = cm.perform_clustering_analysis(preprocessed_df, n_clusters=optimal_k)

In [29]:
# display(kmeans_cluster_labels)

In [ ]:
display(kmeans_cluster_summary)

## !!! PENDING

___
___

# IDEAS

## PRODUCTOS

## TIENDAS

__A priori parece NO TENER sentido aplicar clustering a tiendas con tan pocas muestras.__

In [ ]:
# Creacion caracteristicas para tiendas basadas en el comportamiento temporal de ventas
fc_config = {
    'target_df':'shops',
    'agg_primitives':["sum", "mean", "std", "count"],
    'trans_primitives':[],
    'max_depth':2
}

feature_matrix, feature_defs = fcm.create_features(df_items, df_shops, df_ws, fc_config)

# Revision definiciones caracteristicas creadas
fcm.get_features_descriptions(feature_defs)

# Seleccionar y renombrar características relevantes para el clustering de productos ventas/performance
selected_features_sales = {
    'SUM(sales.units)': 'total_units_sold',
    'SUM(sales.raw_earn)': 'total_revenue',
    'MEAN(sales.units)': 'avg_units_per_week',
    'STD(sales.units)': 'std_units_sold',
    'COUNT(sales)': 'num_sales',
}

feature_matrix = fcm.select_features(feature_matrix, selected_features_sales)

# Remove low information, highly correlated, highly null and single value features
filtered_feature_matrix = fcm.filter_feature_matrix(feature_matrix)

display(filtered_feature_matrix.head(5))

# Renombrar características si es necesario
feature_matrix_renamed = fcm.rename_features(filtered_feature_matrix, selected_features)

display(feature_matrix_renamed.head(5))

# Preprocesamiento datos
scaled_df = pm.preprocess_features(feature_matrix_renamed)

In [ ]:
km_m.plot_elbow_silhouette(scaled_df, 9) # No puede haber mas clusters que etiquetas

In [ ]:
optimal_k = 6 # Segun elbow

kmeans_model, feature_matrix_with_clusters = km_m.apply_kmeans_and_plot(optimal_k, scaled_df, feature_matrix_renamed)

feature_matrix_with_clusters.head(5)

pca_df = km_m.pca_visualization_2d(feature_matrix_with_clusters)

display(pca_df.head())

km_m.pca_variance_plot(feature_matrix_with_clusters)

# Preprocesamos datos
cluster_col='Cluster'
cols_to_scale = feature_matrix_with_clusters.columns.tolist()
df_clusters = feature_matrix_with_clusters[[cluster_col]].copy().reset_index(drop=True)
cols_to_scale.remove(cluster_col) # Columna Objetivo
preprocessed_df = pm.preprocess_features(feature_matrix_with_clusters[cols_to_scale])

# Combinar los DataFrames basándose en la columna 'Index'
preprocessed_df = pd.concat([preprocessed_df, df_clusters], axis=1)

clusters = kmeans_model.labels_

# sales_importance_df = km_m.get_feature_importances(feature_matrix_with_clusters, clusters, imp_threshold=0.05)
sales_importance_df = km_m.get_feature_importances(preprocessed_df, clusters, imp_threshold=0.05)

display(sales_importance_df.head(10))

In [ ]:
display(scaled_df.head())

display(scaled_df.shape)

In [ ]:
km_m.plot_elbow_silhouette(scaled_df, 5) # No puede haber mas clusters que etiquetas

optimal_k = 5 # Segun silueta

kmeans_model, feature_matrix_with_clusters = km_m.apply_kmeans_and_plot(optimal_k, scaled_df, feature_matrix_renamed)

feature_matrix_with_clusters.head(5)

pca_df = km_m.pca_visualization_2d(feature_matrix_with_clusters)

display(pca_df.head())

km_m.pca_variance_plot(feature_matrix_with_clusters)

# Preprocesamos datos
cluster_col='Cluster'
cols_to_scale = feature_matrix_with_clusters.columns.tolist()
df_clusters = feature_matrix_with_clusters[[cluster_col]].copy().reset_index(drop=True)
cols_to_scale.remove(cluster_col) # Columna Objetivo
preprocessed_df = pm.preprocess_features(feature_matrix_with_clusters[cols_to_scale])

# Combinar los DataFrames basándose en la columna 'Index'
preprocessed_df = pd.concat([preprocessed_df, df_clusters], axis=1)

clusters = kmeans_model.labels_

# sales_importance_df = km_m.get_feature_importances(feature_matrix_with_clusters, clusters, imp_threshold=0.05)
sales_importance_df = km_m.get_feature_importances(preprocessed_df, clusters, imp_threshold=0.05)

display(sales_importance_df.head(10))

In [ ]:
# K-means
rf_model_kmeans, preprocessor_kmeans, pca_loadings_kmeans, pca_contributions_kmeans = cm.main(
    feature_matrix_renamed, method='kmeans', n_clusters=2
)

# Este error se da cuando el numero clusters es igual o mayor que el numero de etiquetas!!!